# Table of Contents

- [Connect & Query Data with SQL](#connect--query-data-with-sql)
- [Metadata](#metadata)
- [Hotel Booking Demand & Pricing Analysis](#hotel-booking-demand--pricing-analysis)
    - [Booking Trends & Demand Analysis](#booking-trends--demand-analysis)
        - [Key Insights & Conclusion](#key-insights-conclusion)
    - [Revenue & ADR Analysis](#revenue--adr-analysis)
        - [Key Insights & Conclusion](#key-insights-conclusion)
    - [Cancellation Behavior & Customer Retention](#cancellation-behavior--customer-retention)
        - [Key Insights & Conclusion](#key-insights-conclusion)
    - [Customer Preferences & Special Requests](#customer-preferences--special-requests)
        - [Key Insights & Conclusion](#key-insights-conclusion)
- [Business Recommendations & Strategic Actions](#business-recommendations--strategic-actions)

# Connect & Query Data with SQL

In [4]:
import psycopg2
import pandas as pd

import sys
import os

# Getting the current working directory
current_dir = os.getcwd()

# Adding the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))

# Verifying sys.path to ensure the parent directory is added
print(sys.path)

# Trying importing config
import config

# Step 1: Connecting to the 'hotel_demand_database' using credentials from config.py
conn = psycopg2.connect(
    dbname=config.DB_NAME,
    user=config.DB_USER,
    password=config.DB_PASSWORD,
    host=config.DB_HOST,
    port=config.DB_PORT
)
conn.autocommit = True  # Enabling autocommit so changes are applied automatically
cursor = conn.cursor()

# Step 2: Defining a function to run SQL queries and return results as DataFrame
def run_sql(query):
    """Executes a SQL query and returns a Pandas DataFrame."""
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  # Getting column names
    return pd.DataFrame(rows, columns=columns)

['/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages', '/Users/neda/Portfolio/hotel_pricing', '/Users/neda/Portfolio/hotel_pricing']


# Metadata

**hotelHotel:** H1 = Resort Hotel or H2 = City Hotel

**is_canceled:** Value indicating if the booking was canceled (1) or not (0)

**lead_time:** Number of days that elapsed between the entering date of the booking into the PMS and the arrival date

**arrival_date_year:** Year of arrival date

**arrival_date_month:** Month of arrival date

**arrival_date_week_number:** Week number of year for arrival date

**arrival_date_day_of_month:** Day of arrival date

**stays_in_weekend_nights:** Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel

**stays_in_week_nights:** Number of weeknights (Monday to Friday) the guest stayed or booked to stay at the hotel

**adults:** Number of adults

**children:** Number of children

**babies:** Number of babies

**meal:** Type of meal booked. Categories are presented in standard hospitality meal packages: Undefined/SC – no meal package; BB – Bed & Breakfast; HB – Half board (breakfast and one other meal – usually dinner); FB – Full board (breakfast, lunch and dinner)

**country:** Country of origin. Categories are represented in the ISO 3155–3:2013 format

**market_segment:** Market segment designation. In categories, the term “TA” means “Travel Agents” and “TO” means “Tour Operators”

**distribution_channel:** Booking distribution channel. The term “TA” means “Travel Agents” and “TO” means “Tour Operators”

**is_repeated_guest:** Value indicating if the booking name was from a repeated guest (1) or not (0)

**previous_cancellations:** Number of previous bookings that were cancelled by the customer prior to the current booking

**previous_bookings_not_canceled:** Number of previous bookings not cancelled by the customer prior to the current booking

**reserved_room_type:** Code of the room type reserved. Code is presented instead of designation for anonymity reasons.

**assigned_room_type:** Code for the type of room assigned to the booking. Sometimes, the assigned room type differs from the reserved room type due to hotel operation reasons (e.g. overbooking) or by customer request. Code is presented instead of designation for anonymity reasons.

**booking_changes:** Number of changes/amendments made to the booking from the moment the booking was entered on the PMS until the moment of check-in or cancellation

**deposit_type:** Indication of whether the customer deposited to guarantee the booking. This variable can assume three categories: No Deposit, Non-Refund, and Refundable.

**agent:** ID of the travel agency that made the booking

**company:** ID of the company/entity that made the booking or responsible for paying the booking. ID is presented instead of designation for anonymity reasons

**days_in_waiting_list:** Number of days the booking was in the waiting list before it was confirmed to the customer

**customer_type:** Type of booking, assuming one of four categories: Contract - when the booking has an allotment or other type of contract associated to it; Group – when the booking is associated to a group; Transient – when the booking is not part of a group or contract, and is not associated to other transient booking; Transient-party – when the booking is transient, but is associated to at least other transient booking

**adr:** Average Daily Rate is defined by dividing the sum of all lodging transactions by the total number of staying nights

**required_car_parking_spaces:** Number of car parking spaces required by the customer

**total_of_special_requests:** Number of special requests made by the customer (e.g. twin bed or high floor)

**reservation_status:** Reservation last status, assuming one of three categories: Canceled – booking was canceled by the customer; Check-Out – customer has checked in but already departed; No-Show – customer did not check-in and did inform the hotel of the reason why

**reservation_status_date:** The Date at which the last status was set. This variable can be used in conjunction with the Reservation Status to understand when the booking was canceled or when the customer checked out of the hotel

<span style="font-size: 23px;">**Data Cleaning & Preparation**

The dataset has already undergone a comprehensive cleaning and preparation process before being inserted into the database for further analysis.

Data cleaning & preparation involved several critical steps to ensure the dataset's accuracy and integrity. We assessed and corrected inconsistencies, categorizing them into three groups: Critical, Inconsistency, and Anomaly detection. These issues were addressed based on their severity and overall business logic. Missing values were either removed or imputed depending on the context. Additionally, strict duplicate records were identified and removed to maintain data uniqueness and reliability. To prevent skewed analysis, potential outliers were detected and appropriately handled.

All data cleaning steps have been thoroughly documented in a separate notebook within this directory: Data_Cleaning_and_Preparation.


# Hotel Booking Demand & Pricing Analysis

Understanding hotel booking data is crucial for optimizing revenue, improving customer retention, and enhancing operational efficiency. By analyzing key factors such as demand patterns, revenue trends, cancellation behavior, and guest preferences, hotels can make data-driven decisions to maximize profitability and guest satisfaction. For instance, identifying seasonality trends helps in dynamic pricing, while analyzing cancellation patterns allows for better overbooking strategies. Insights into customer preferences also enable hotels to tailor their services, ultimately improving guest experiences and increasing repeat bookings.

To explore these aspects comprehensively, we will analyze the data from four key perspectives:

- Booking Trends & Demand Analysis – Examining seasonality, lead times, and occupancy trends to understand peak periods and demand fluctuations.
- Revenue & Average Daily Rate (ADR) Analysis – Evaluating revenue generation, ADR patterns, and their relationship with different customer segments.
- Cancellation Behavior & Customer Retention – Investigating cancellation trends, their impact on revenue, and factors influencing customer loyalty.
- Customer Preferences & Special Requests – Analyzing room type preferences, special requests, and their correlation with guest satisfaction and repeat bookings.

This structured analysis will provide actionable insights for revenue management, customer segmentation, and overall business strategy.

**hotel_demand_adr table:**

In [5]:
run_sql("""
SELECT * FROM hotel_demand_adr
""")

,booking_id,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,86977,City Hotel,0,23,2016,April,15,9,2,1,...,0,No Deposit,7,0,Transient,93.82,0,2,Check-Out,2016-04-12
1,86978,City Hotel,0,81,2016,April,16,10,2,0,...,0,No Deposit,9,0,Transient,85.50,0,1,Check-Out,2016-04-12
2,86979,City Hotel,0,4,2016,April,15,8,2,2,...,0,No Deposit,9,0,Transient,112.50,0,1,Check-Out,2016-04-12
3,86980,City Hotel,0,17,2016,April,16,10,2,0,...,2,No Deposit,7,0,Transient,97.41,0,3,Check-Out,2016-04-12
4,86981,City Hotel,0,7,2016,April,15,9,2,1,...,0,No Deposit,8,0,Transient,133.83,0,2,Check-Out,2016-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86477,86972,City Hotel,0,73,2016,April,15,9,2,1,...,0,No Deposit,9,0,Transient,115.80,0,1,Check-Out,2016-04-12
86478,86966,City Hotel,0,60,2016,April,15,8,2,2,...,0,No Deposit,28,0,Transient-Party,75.00,0,1,Check-Out,2016-04-12
86479,86973,City Hotel,0,25,2016,April,16,11,1,0,...,0,No Deposit,8,0,Transient,101.00,0,1,Check-Out,2016-04-12
86480,86975,City Hotel,0,1,2016,April,15,8,2,2,...,0,No Deposit,9,0,Transient,116.00,0,1,Check-Out,2016-04-12


## Booking Trends & Demand Analysis

<span style="font-size: 19px;">Date Limits of the Data:

In [52]:
run_sql("""
SELECT 
    MIN(arrival_date_year) AS min_year, 
    MIN(arrival_date_month) AS min_month,
    MAX(arrival_date_year) AS max_year, 
    MAX(arrival_date_month) AS max_month
FROM hotel_demand_adr
;""")

,min_year,min_month,max_year,max_month
0,2015,April,2017,September


<span style="font-size: 16px;">**Result:**

Date-wise, data starts from April 2015 and ends in September 2017.

<span style="font-size: 19px;">Monthly booking trends:

In [9]:
run_sql("""
SELECT arrival_date_month, COUNT(*) 
FROM hotel_demand_adr
GROUP BY arrival_date_month
ORDER BY COUNT(*) DESC
;""")

,arrival_date_month,count
0,August,11104
1,July,9959
2,May,8272
3,April,7849
4,June,7727
5,March,7445
6,October,6886
7,September,6603
8,February,6027
9,December,5063


<span style="font-size: 16px;">**Result:**

The data shows a clear seasonal pattern in monthly bookings, with the highest number of bookings occurring in August (11,104) and gradually decreasing towards the start of the year, where January sees the lowest number (4,607). There is a noticeable peak during the summer months, particularly in July and August, suggesting that this period is a popular time for bookings. Following this, the number of bookings consistently declines through the fall and into winter, with a significant drop observed from August to December. This could indicate that demand for bookings tends to be lower in the colder months, possibly due to reduced travel or leisure activities during the winter season.

<span style="font-size: 19px;"> Weekday vs. Weekend stays:

In [16]:
run_sql("""
SELECT Count(*) as Total_Bookings,
       SUM(stays_in_week_nights) AS weekday_stays, 
       SUM(stays_in_weekend_nights) AS weekend_stays
FROM hotel_demand_adr
;""")

,total_bookings,weekday_stays,weekend_stays
0,86482,223991,85806


<span style="font-size: 16px;">**Result:**

Definition Review: 
- total_bookings: Represents the total number of bookings in the dataset.
- weekday_stays: The total number of nights that bookings are made for weekdays.
- weekend_stays: The total number of nights that bookings are made for weekends.

Weekdays: For the 223,991 weekday stay nights, this represents stays over 5 days a week. To adjust, we can normalize the weekday stays as a proportion of these 5 days; 223991/5, which equals 44,798 stays/week. 

Similarly, for the 85,806 weekend stay nights, which represents 2 days a week, we can adjust for weeks: Weekend stays: 42,903 stays/week

**Distribution of Weekday vs. Weekend Stays**

The distribution between weekday and weekend stays shows a relatively balanced pattern, with weekday stays slightly outnumbering weekend stays. Specifically, there are 44,798 weekday stays per week compared to 42,903 weekend stays per week. This indicates that, on average, there are just a few more weekday bookings than weekend bookings. However, this difference is not drastic, suggesting that guests tend to book both types of stays fairly consistently.                                             

<span style="font-size: 19px;"> Year-over-year growth in bookings:

the Year-Over-Year (YoY) growth compares a company's performance in its most recent month to its numbers from the same month in the previous year.

In [89]:
run_sql("""
WITH aggregated_data AS (
    SELECT 
        arrival_date_year,
        arrival_date_month,
        COUNT(*) AS current_year_month
    FROM hotel_demand_adr
    GROUP BY arrival_date_year, arrival_date_month
)
SELECT 
    arrival_date_year,
    arrival_date_month,
    current_year_month,
    LAG(current_year_month) OVER (PARTITION BY arrival_date_month ORDER BY arrival_date_year) AS last_year_month,
    
    -- Calculating YOY with COALESCE to handle NULL values
    (CAST(COALESCE(current_year_month, 0) AS FLOAT) - 
    CAST(COALESCE(LAG(current_year_month) OVER (PARTITION BY arrival_date_month ORDER BY arrival_date_year), 0) AS FLOAT))
    / NULLIF(CAST(COALESCE(LAG(current_year_month) OVER (PARTITION BY arrival_date_month ORDER BY arrival_date_year), 0) AS FLOAT), 0) * 100 AS yoy

FROM aggregated_data
ORDER BY arrival_date_year, arrival_date_month
;""")

,arrival_date_year,arrival_date_month,current_year_month,last_year_month,yoy
0,2015,August,2437,NaN,NaN
1,2015,December,1968,NaN,NaN
2,2015,July,1662,NaN,NaN
3,2015,November,1655,NaN,NaN
4,2015,October,2688,NaN,NaN
5,2015,September,2823,NaN,NaN
6,2016,April,3730,NaN,NaN
7,2016,August,4381,2437.0,79.770209
8,2016,December,3095,1968.0,57.266260
9,2016,February,2772,NaN,NaN


**Key Observations:**

- The Year-over-Year (YOY) values for 2015 are NaN because there is no data for the previous year (2014). This is expected behavior, as there is no "last year" data to compare it with.

- YOY for 2016 and 2017: The YOY values are calculated where there is data for the same month in the previous year. The values indicate the percentage change in the count of arrivals compared to the same month in the prior year. For example: 2016 August (4381) compared to 2015 August (2437) results in a YOY of 79.77%.

- High YOY Variability: The YOY values fluctuate significantly between months.

**Trends:**

In 2017, there are fewer YOY increases compared to 2016, such as in August (-2.17%) and March (-4.75%), which could indicate a slight decrease in overall demand or changes in external factors that affected arrivals. The YOY for May across years shows a positive trend (e.g., 22.13% for 2017 vs. 2016), suggesting a steady increase in arrivals during this month.

**Insights:**

Strong growth in 2016 compared to 2015: The arrivals in 2016 seem to have significantly increased in many months compared to 2015, with large YOY increases, such as 79.77% in August and 129.30% in July.

Some months have decreasing trends in 2017: In 2017, months like August and March show a slight decline compared to the previous year, with YOY values of -2.17% and -4.75%.

General trend of growth overall: Despite the occasional dips, the data generally shows an upward trend in arrivals over time.

**Recommendations:**

Further Analysis of Seasonality: performing additional analysis on the monthly trends (e.g., aggregating over several years) to identify strong seasonal patterns that could inform future planning.

Impact of Decreases in 2017: Investigating the factors that led to the decreases in 2017 (such as August and March) could provide insights into potential issues, such as economic or external factors, that may have impacted demand in those months.

<span style="font-size: 19px;"> Highest & Lowest Demand Periods:

In [55]:
run_sql("""
SELECT arrival_date_month, COUNT(*) AS highest_total_bookings
FROM hotel_demand_adr
GROUP BY arrival_date_month
ORDER BY total_bookings DESC
LIMIT 1;
""")

,arrival_date_month,total_bookings
0,August,11104


In [56]:
run_sql("""
SELECT arrival_date_month, COUNT(*) AS lowest_total_bookings
FROM hotel_demand_adr
GROUP BY arrival_date_month
ORDER BY total_bookings
LIMIT 1;
""")

,arrival_date_month,total_bookings
0,January,4607


<span style="font-size: 19px;"> Weekend vs. Weekday Stay Ratios Per Year 

In [59]:
run_sql("""
SELECT arrival_date_year, 
       SUM(stays_in_weekend_nights) AS weekend_stays, 
       SUM(stays_in_week_nights) AS weekday_stays,
       ROUND((SUM(stays_in_weekend_nights) * 100.0 / NULLIF(SUM(stays_in_week_nights), 0)), 2) AS weekend_weekday_ratio
FROM hotel_demand_adr
GROUP BY arrival_date_year
ORDER BY arrival_date_year;
""")

,arrival_date_year,weekend_stays,weekday_stays,weekend_weekday_ratio
0,2015,13543,35882,37.74
1,2016,40423,104753,38.59
2,2017,31840,83356,38.20


<span style="font-size: 16px;">**Result:**


The weekend vs. weekday stay ratios have remained consistent over the years, averaging around 37.

<span style="font-size: 19px;"> Average Stay Duration Over Time:

In [76]:
run_sql("""
SELECT arrival_date_year, arrival_date_month, 
       AVG(stays_in_week_nights + stays_in_weekend_nights) AS avg_stay_duration
FROM hotel_demand_adr
GROUP BY arrival_date_year, arrival_date_month
ORDER BY arrival_date_year, arrival_date_month;
""")

,arrival_date_year,arrival_date_month,avg_stay_duration
0,2015,August,4.2798522773902339
1,2015,December,3.0508130081300813
2,2015,July,4.7262334536702768
3,2015,November,3.2422960725075529
4,2015,October,3.2775297619047619
5,2015,September,3.8823946156571024
6,2016,April,3.2772117962466488
7,2016,August,4.0479342615841132
8,2016,December,3.3915993537964459
9,2016,February,2.9246031746031746


<span style="font-size: 16px;">**Result:**

The average stay duration fluctuates across months and years, with August 2015 having the longest stay duration at 4.28 nights. In general, stay durations tend to be slightly shorter at the beginning of each year, with January 2016 showing the lowest at 2.68 nights. The highest stay durations in 2017 were seen in August (4.16 nights) and July (4.07 nights). This suggests that guests may extend their stays during the summer months, possibly due to vacation or holiday periods.

<span style="font-size: 19px;"> Most Popular Booking Month for Each Year:

In [63]:
run_sql("""
SELECT arrival_date_year, arrival_date_month, total_bookings, rank
FROM (
    SELECT arrival_date_year, arrival_date_month, COUNT(*) AS total_bookings,
           RANK() OVER (PARTITION BY arrival_date_year ORDER BY COUNT(*) DESC) AS rank
    FROM hotel_demand_adr
    GROUP BY arrival_date_year, arrival_date_month
) subquery
WHERE rank in (1, 2);
""")

,arrival_date_year,arrival_date_month,total_bookings,rank
0,2015,September,2823,1
1,2015,October,2688,2
2,2016,August,4381,1
3,2016,October,4198,2
4,2017,May,4548,1
5,2017,July,4486,2


<span style="font-size: 16px;">**Result:**

The highest bookings in each year are concentrated around May 2017 and August 2016. Specifically, May 2017 leads with 4,548 bookings, followed by July 2017 with 4,486 bookings. The rankings demonstrate peak booking periods in these months, indicating either seasonal spikes or successful marketing strategies during those months or both.

<span style="font-size: 19px;"> Yearly Booking Trends by Market Segment:

In [64]:
run_sql("""
SELECT arrival_date_year, market_segment, COUNT(*) AS total_bookings
FROM hotel_demand_adr
GROUP BY arrival_date_year, market_segment
ORDER BY arrival_date_year, total_bookings DESC;
;""")

,arrival_date_year,market_segment,total_bookings
0,2015,Online TA,5683
1,2015,Offline TA/TO,3108
2,2015,Direct,2126
3,2015,Groups,1329
4,2015,Corporate,836
5,2015,Complementary,149
6,2015,Undefined,2
7,2016,Online TA,25179
8,2016,Offline TA/TO,6988
9,2016,Direct,5258


<span style="font-size: 16px;">**Result:**

The distribution of bookings across market segments shows a shift towards Online Travel Agents (Online TA), which has the highest bookings in all years. For instance, Online TA recorded 5,683 bookings in 2015, increasing to 25,179 bookings in 2016, and further rising to 20,429 bookings in 2017. Other segments, like Offline TA/TO and Direct, also show steady activity, but Online TA clearly dominates, signifying the importance of online platforms in booking patterns.

<span style="font-size: 19px;"> Monthly Cancellation Rate Over Time:

In [78]:
run_sql("""
SELECT arrival_date_year, SUM(is_canceled)::FLOAT / COUNT(*) * 100 AS cancelation_rate
FROM hotel_demand_adr
GROUP BY arrival_date_year, arrival_date_month
;""")

,arrival_date_year,cancelation_rate
0,2015,16.331845
1,2017,26.758833
2,2016,26.916890
3,2015,23.348379
4,2015,18.699187
5,2015,14.561934
6,2017,36.934204
7,2015,30.625752
8,2017,25.545617
9,2016,23.183845


<span style="font-size: 16px;">**Result:**

The cancellation rates vary significantly across years, with 2017 showing a higher frequency of cancellations, reaching a peak of 36.93% in May 2017. The cancellation rates in 2015 are generally lower, with the highest rate being 30.63% in September 2015. The trends indicate that, over time, cancellations have become more prevalent, possibly due to changing market conditions or customer behavior.

<span style="font-size: 19px;"> Peak Lead Time Booking Periods: 

In [79]:
run_sql("""
SELECT arrival_date_year, 
       PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY lead_time) AS p90_lead_time
FROM hotel_demand_adr
GROUP BY arrival_date_year
ORDER BY arrival_date_year
;""")

,arrival_date_year,p90_lead_time
0,2015,159.0
1,2016,193.0
2,2017,223.0


<span style="font-size: 16px;">**Result:**

For 2015, the 90th percentile lead time is 159 days, meaning 90% of the bookings were made 159 days or less in advance, and only 10% of the bookings were made more than 159 days in advance.

The data indicates that, by 2017, a significant portion of bookings were made with much longer lead times compared to 2015, reflecting changing booking behaviors, perhaps due to increased planning or advanced booking policies.

<span style="font-size: 19px;"> Lead Time Trends Over Time:

In [80]:
run_sql("""
SELECT arrival_date_year, arrival_date_month, AVG(lead_time) AS avg_lead_time
FROM hotel_demand_adr
GROUP BY arrival_date_year, arrival_date_month
ORDER BY arrival_date_year, arrival_date_month
;""")

,arrival_date_year,arrival_date_month,avg_lead_time
0,2015,August,58.8682806729585556
1,2015,December,38.6712398373983740
2,2015,July,79.4241877256317690
3,2015,November,35.5697885196374622
4,2015,October,57.8642113095238095
5,2015,September,74.0807651434643996
6,2016,April,58.4670241286863271
7,2016,August,105.5599178269801415
8,2016,December,70.4012924071082391
9,2016,February,25.7438672438672439


<span style="font-size: 16px;">**Result:**

Lead time trends reveal seasonal variations across months. The data suggests that over the years, there was an increasing trend in how far in advance customers were booking, with peak months like July 2017 showing the highest lead time, potentially indicating a shift toward longer booking windows during high-demand periods.

### Key Insights & Conclusion

<span style="font-size: 18px;">Seasonality and Booking Trends: 

The dataset reveals clear seasonal patterns in demand, with bookings peaking in the summer months (particularly in July and August), followed by a significant decline during the winter months. These seasonal shifts suggest that the market is highly sensitive to external factors, such as weather, holidays, and travel preferences, which drive the bulk of demand during warmer months. The steep drop from August to December suggests that winter months see lower demand, likely due to reduced leisure travel and fewer vacation opportunities.

<span style="font-size: 18px;">Impact of Market Segments and Booking Channels: 

A clear shift towards Online Travel Agents (OTA) is observed, which consistently shows an increase in bookings across years. This reflects the growing importance of digital channels in customer decision-making and booking behavior. In contrast, segments such as Offline TA/TO and Direct bookings have seen slower growth, indicating that customers are increasingly leveraging online platforms for better deals, flexibility, and convenience. This shift highlights the need for strong online marketing strategies and partnerships with OTAs to capture a larger share of the market.

<span style="font-size: 18px;">Bookings and Cancellation Trends: 

While overall bookings have grown over time, a noticeable increase in cancellation rates in 2017, particularly in peak months like May, suggests that customer behavior is changing. The cancellation rate in May 2017 peaked at 36.93%, the highest recorded, indicating that customers are becoming more comfortable with canceling their reservations or perhaps taking advantage of flexible cancellation policies. This trend could be influenced by factors such as economic uncertainty, external events, or a shift in customer expectations around booking flexibility. This trend warrants further investigation to understand its full impact and identify potential causes, such as market competition, economic downturns, or changes in customer preferences.

<span style="font-size: 18px;">Weekend vs. Weekday Stay Distribution: 

The analysis of weekday vs. weekend stays reveals a balanced distribution, with a slight dominance of weekday bookings over weekends. This consistent pattern throughout the years suggests that guests are booking stays fairly evenly across both weekdays and weekends. The similarity between the two types of stays also implies that guests are not highly dependent on weekends for their stays, and businesses could leverage this pattern by targeting guests across both types of bookings. Given the similar number of weekday and weekend stays, a flexible pricing strategy that adjusts based on the demand during these periods could maximize revenue.

<span style="font-size: 18px;">Stay Duration Trends: 

The average stay duration fluctuates over the course of the year, with the longest stays typically seen during the summer months. This suggests that longer stays correlate with vacation periods or holiday travel, while shorter stays are more common during the start of the year when travel activity is relatively low. This pattern provides insights into customer preferences, where vacationers tend to extend their stays in peak seasons, whereas business travelers or weekend getaways may prefer shorter stays. Understanding these trends can help hotels optimize their pricing strategies and package deals to appeal to customers during peak seasons.

<span style="font-size: 18px;">Lead Time Trends: 

There is a noticeable trend toward longer booking lead times as peak seasons approach. For example, the summer months of July and August show increased lead times, possibly due to higher demand during these periods. This indicates a shift towards more advanced planning for popular travel months, which could be attributed to a combination of higher demand and customers securing reservations earlier to avoid price hikes or availability issues. Hotels could take advantage of this trend by offering early-bird promotions or discounts for advanced bookings to capture more guests early on.

<span style="font-size: 21px;">**Overall Business Recommendations**

- Maximize Online Presence
- Address Cancellations
- Target Seasonal Demand
- Optimize Weekend & Weekday Pricing
- Promote Extended Stays During Peak Seasons
- Leverage Booking Lead Times

## Revenue & ADR Analysis

<span style="font-size: 19px;"> Monthly ADR Trends:

In [81]:
run_sql("""
SELECT arrival_date_month, AVG(adr) AS avg_adr
FROM hotel_demand_adr
GROUP BY arrival_date_month
ORDER BY avg_adr DESC;
""")

,arrival_date_month,avg_adr
0,August,149.983900
1,July,135.275868
2,June,119.833059
3,September,112.430201
4,May,111.328899
5,April,103.759070
6,October,90.405170
7,March,81.200592
8,December,81.083559
9,February,74.730435


<span style="font-size: 16px;">**Result:**

- Peak Pricing in August & July:

ADR peaks in August (149.98 dollars) and July (135.28 dollars), reflecting the high demand during summer.
This suggests that these months coincide with peak tourist seasons, where demand drives higher prices.

- Off-Peak Pricing in Winter:

January (70.18 dollars) and November (72.96 dollars) have the lowest ADRs, indicating a slower season.
Hotels likely offer lower rates to attract customers when demand is low.

If a hotel wants to maximize revenue, it should focus on dynamic pricing strategies, increasing rates aggressively during peak months while offering strategic promotions in off-peak months to maintain occupancy.

<span style="font-size: 19px;">  ADR Trends over Time:

In [82]:
run_sql("""
SELECT arrival_date_year, arrival_date_month, AVG(adr) AS avg_adr
FROM hotel_demand_adr
GROUP BY arrival_date_year, arrival_date_month
ORDER BY arrival_date_year, arrival_date_month;
""")

,arrival_date_year,arrival_date_month,avg_adr
0,2015,August,123.146672
1,2015,December,71.740640
2,2015,July,112.352046
3,2015,November,59.257498
4,2015,October,79.170867
5,2015,September,100.199476
6,2016,April,91.185298
7,2016,August,148.325490
8,2016,December,87.024388
9,2016,February,69.901006


<span style="font-size: 16px;">**Result:**

ADR is increasing over time: August ADR rose from 123.15 dollars (2015) → 148.33 dollars (2016) → 166.94 dollars (2017), showing a consistent upward trend.

Seasonal consistency: Every year, the summer months maintain the highest ADRs, while winter remains low.

This suggests strong market demand growth and possible inflation or increased tourism. Hotels can focus on premium pricing strategies during high-demand months.

<span style="font-size: 19px;"> Impact of Market Segment on ADR:

In [83]:
run_sql("""
SELECT market_segment, AVG(adr) AS avg_adr
FROM hotel_demand_adr
GROUP BY market_segment
ORDER BY avg_adr DESC;
""")

,market_segment,avg_adr
0,Online TA,117.982554
1,Direct,115.965458
2,Aviation,100.613628
3,Offline TA/TO,81.524485
4,Groups,75.106557
5,Corporate,68.311020
6,Undefined,15.000000
7,Complementary,3.166524


<span style="font-size: 16px;">**Result:**

Online Travel Agencies (OTA) and Direct Bookings have the highest ADRs (118 dollars): Likely due to higher pricing flexibility and last-minute bookings at premium rates.

Corporate (68.31 dollars) and Group Bookings (75.10 dollars) have lower ADRs: May suggest bulk discounts or negotiated rates for business and large groups.

Complementary bookings (3.17 dollars) and Undefined (15.00 dollars) are almost negligible: Likely promotional stays or system anomalies.

Hotels can maximize revenue by encouraging direct bookings and leveraging OTA platforms for premium pricing.

<span style="font-size: 19px;"> Impact of Hotel Type on ADR:

In [84]:
run_sql("""
SELECT hotel, AVG(adr) AS avg_adr
FROM hotel_demand_adr
GROUP BY hotel
ORDER BY avg_adr DESC;
""")

,hotel,avg_adr
0,City Hotel,111.156206
1,Resort Hotel,98.526128


<span style="font-size: 16px;">**Result:**

City Hotels charge higher than Resort Hotels: Likely due to business travelers willing to pay more for urban convenience.
                                                                                                  
Resorts may rely on longer stays for profitability, while city hotels can focus on high turnover and premium services.

<span style="font-size: 19px;"> Revenue Contribution by Country:

In [87]:
run_sql("""
SELECT country, SUM(adr) AS revenue
FROM hotel_demand_adr
GROUP BY country
ORDER BY revenue DESC
LIMIT 10;
""")

,country,revenue
0,PRT,2605262.74
1,GBR,1010911.03
2,FRA,989461.76
3,ESP,876582.77
4,DEU,568806.78
5,ITA,354449.01
6,IRL,301772.60
7,BEL,237002.85
8,USA,233094.38
9,BRA,223871.54


<span style="font-size: 16px;">**Result:**

Portugal (PRT) - The largest revenue contributor, likely due to domestic tourism and a strong local customer base.
Hotels should maintain a localized marketing approach and loyalty programs for domestic travelers.

United Kingdom (GBR) - A significant source of revenue, possibly due to strong historical ties, ease of travel, and high spending capacity of British tourists.
Targeted promotions for UK travelers, especially during peak travel seasons.

France (FRA) - Close third, suggesting steady inbound tourism from France.
Offering French-language services, customized travel packages, and targeted digital campaigns could enhance revenue.

<span style="font-size: 19px;"> Year-over-Year ADR Growth Rate:

In [90]:
run_sql("""
WITH aggregated_data AS (
    SELECT 
        arrival_date_year,
        arrival_date_month,
        SUM(adr) AS adr_current_year_month
    FROM hotel_demand_adr
    GROUP BY arrival_date_year, arrival_date_month
)
SELECT 
    arrival_date_year,
    arrival_date_month,
    adr_current_year_month,
    LAG(adr_current_year_month) OVER (PARTITION BY arrival_date_month ORDER BY arrival_date_year) AS adr_last_year_month,
    
    -- Calculating YOY with COALESCE to handle NULL values
    (CAST(COALESCE(adr_current_year_month, 0) AS FLOAT) - 
    CAST(COALESCE(LAG(adr_current_year_month) OVER (PARTITION BY arrival_date_month ORDER BY arrival_date_year), 0) AS FLOAT))
    / NULLIF(CAST(COALESCE(LAG(adr_current_year_month) OVER (PARTITION BY arrival_date_month ORDER BY arrival_date_year), 0) AS FLOAT), 0) * 100 AS yoy

FROM aggregated_data
ORDER BY arrival_date_year, arrival_date_month
;""")

,arrival_date_year,arrival_date_month,adr_current_year_month,adr_last_year_month,yoy
0,2015,August,300108.44,NaN,NaN
1,2015,December,141185.58,NaN,NaN
2,2015,July,186729.10,NaN,NaN
3,2015,November,98071.16,NaN,NaN
4,2015,October,212811.29,NaN,NaN
5,2015,September,282863.12,NaN,NaN
6,2016,April,340121.16,NaN,NaN
7,2016,August,649813.97,300108.44,116.526390
8,2016,December,269340.48,141185.58,90.770531
9,2016,February,193765.59,NaN,NaN


<span style="font-size: 16px;">**Result:**

ADR (Average Daily Rate) has generally increased year over year, with some months seeing significant jumps. Notably, July 2016 saw a 168% YoY increase, November 2016 grew by 167%, and January 2017 surged by 85%. These indicate either stronger pricing power or shifts in demand dynamics, possibly due to seasonality, economic conditions, or marketing strategies.

<span style="font-size: 19px;">  Percentage Change in Average ADR each Year:

In [95]:
run_sql("""
SELECT arrival_date_year, 
       AVG(adr), 2 AS avg_adr,
       LAG(AVG(adr)) OVER (ORDER BY arrival_date_year) AS prev_year_adr,
       ((AVG(adr) - LAG(AVG(adr)) OVER (ORDER BY arrival_date_year)) / NULLIF(LAG(AVG(adr)) OVER (ORDER BY arrival_date_year), 0)) * 100 AS percentage_yearly_increase
FROM hotel_demand_adr
GROUP BY arrival_date_year
ORDER BY arrival_date_year;
""")

,arrival_date_year,avg,avg_adr,prev_year_adr,percentage_yearly_increase
0,2015,92.327416,2,NaN,NaN
1,2016,101.461621,2,92.327416,9.893275
2,2017,118.508592,2,101.461621,16.801398


<span style="font-size: 16px;">**Result:** 

ADR grew 9.89% from 2015 to 2016 and 16.8% from 2016 to 2017, showing an accelerating upward trend. This suggests the hotel's pricing power strengthened, potentially due to higher demand, better revenue management, or inflationary pressures.

<span style="font-size: 19px;">  Effect of Lead Time on ADR:

In [96]:
run_sql("""
SELECT 
CASE 
    WHEN lead_time BETWEEN 0 AND 7 THEN '0-7 days'
    WHEN lead_time BETWEEN 8 AND 30 THEN '8-30 days'
    WHEN lead_time BETWEEN 31 AND 90 THEN '1-3 Months'
    WHEN lead_time BETWEEN 90 AND 180 THEN '3-6 Months'
    WHEN lead_time BETWEEN 180 AND 365 THEN '6-12 Months'
    ELSE '1+ Year'
END AS lead_time_range,
AVG(adr) AS avg_adr
FROM hotel_demand_adr
GROUP BY lead_time_range
ORDER BY avg_adr DESC;
""")

,lead_time_range,avg_adr
0,3-6 Months,113.835332
1,1-3 Months,110.681129
2,8-30 days,110.291407
3,6-12 Months,103.696753
4,0-7 days,91.572860
5,1+ Year,82.530724


<span style="font-size: 16px;">**Result:** 

Longer lead times tend to correlate with lower ADR—bookings made more than a year in advance have the lowest ADR (82.53 dollars), whereas bookings made within 3-6 months command the highest ADR (113.83 dollars). This suggests price-sensitive customers book early, while shorter lead times capture higher-spending guests.

<span style="font-size: 19px;">  Revenue Lost Due to Cancellations:

In [106]:
run_sql("""
SELECT SUM(CASE WHEN is_canceled = 1 THEN adr ELSE 0 END) AS lost_revenue,
       SUM(CASE WHEN is_canceled = 1 THEN adr ELSE 0 END)/SUM(adr) * 100 AS lost_revenue_percentage    
FROM hotel_demand_adr;
""")

,lost_revenue,lost_revenue_percentage
0,2796022.26,30.431126


<span style="font-size: 16px;">**Result:**

A staggering 2.79 million dollars in lost revenue represents nearly 30% of potential earnings. This suggests issues with cancellations, no-shows, or inefficient booking policies. Addressing this could substantially boost total revenue.

<span style="font-size: 19px;">  Most Profitable Room Types:

In [99]:
run_sql("""
SELECT reserved_room_type, 
       SUM(adr) AS total_revenue
FROM hotel_demand_adr
GROUP BY reserved_room_type
ORDER BY total_revenue DESC
LIMIT 3;
""")

,reserved_room_type,total_revenue
0,A,5181446.64
1,D,2115874.20
2,E,755026.45


<span style="font-size: 16px;">**Result:** 

Room type A generates the highest revenue (5.18M dollars), more than double that of the second-highest, type D. This indicates either higher demand, pricing, or availability for type A, while room type E, at 755K dollars, suggests either limited inventory or lower appeal.

<span style="font-size: 19px;"> Outlier Detection in ADR:

In [135]:
run_sql("""
WITH adr_distribution AS (
    SELECT 
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY adr) AS Q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY adr) AS Q3
    FROM hotel_demand_adr
)
SELECT * 
FROM hotel_demand_adr
WHERE adr < (SELECT Q1 - 1.5 * (Q3 - Q1) FROM adr_distribution)
   OR adr > (SELECT Q3 + 1.5 * (Q3 - Q1) FROM adr_distribution)
ORDER BY adr DESC;
""")

,booking_id,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,26682,Resort Hotel,0,2,2016,August,33,8,1,0,...,0,No Deposit,250,0,Transient,335.00,0,0,Check-Out,2016-08-09
1,112700,City Hotel,0,35,2017,May,21,25,0,3,...,1,No Deposit,14,0,Transient,335.00,0,0,Check-Out,2017-05-28
2,112707,City Hotel,0,14,2017,May,21,25,0,3,...,0,No Deposit,14,0,Transient,335.00,0,0,Check-Out,2017-05-28
3,14433,Resort Hotel,0,25,2017,August,34,22,0,3,...,1,No Deposit,-1,0,Transient,335.00,0,2,Check-Out,2017-08-25
4,39277,Resort Hotel,0,84,2017,August,32,8,0,5,...,0,No Deposit,-1,0,Transient,335.00,0,1,Check-Out,2017-08-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,72714,City Hotel,1,158,2017,August,31,3,2,5,...,0,No Deposit,9,0,Transient,226.80,0,0,Canceled,2017-05-30
2366,72943,City Hotel,1,184,2017,August,32,10,0,3,...,1,No Deposit,9,0,Transient,226.80,0,0,Canceled,2017-06-12
2367,7693,Resort Hotel,1,100,2016,August,34,20,4,7,...,1,No Deposit,240,0,Transient,226.73,0,0,Canceled,2016-05-23
2368,20444,Resort Hotel,0,4,2017,June,26,30,1,2,...,0,No Deposit,240,0,Transient,226.67,0,2,Check-Out,2017-07-03


### Key Insights & Conclusion

<span style="font-size: 18px;">Seasonality and Demand Fluctuations:

Peak Pricing: The highest ADRs occur during the summer months, specifically in August and July, where prices are elevated to capitalize on peak tourism demand. In August 2017, ADR peaked at 149.98 dollars, while July 2017 followed closely with 135.28 dollars. This supports the assumption that these months are associated with high-demand periods, likely due to vacationers and tourists seeking accommodation.

Off-Peak Pricing: Conversely, the winter months, such as January and November, show the lowest ADRs (e.g., 70.18 dollars in January 2017), suggesting that demand significantly drops during the off-season. This lower pricing is a strategic approach to attract guests during slower periods.

<span style="font-size: 18px;">Long-Term ADR Growth:

ADR has been consistently rising year over year, with 2017 showing the most significant increase. For instance, August 2017 saw an increase from 123.15 dollars (2015) to 166.94 dollars (2017), marking an upward trajectory. This suggests that the overall market demand has grown, possibly due to increased tourism, improved marketing, or even inflationary pressures.
The 9.89% increase from 2015 to 2016 and 16.8% growth from 2016 to 2017 further indicate that hotels are successfully capitalizing on increased demand, enhancing their pricing strategies over time.

<span style="font-size: 18px;">Market Segments and Revenue Opportunities:

Online Travel Agencies (OTA) and Direct bookings are the most profitable channels, with higher ADRs of 118 dollars compared to Corporate and Group bookings at 68.31 dollars and 75.10 dollars, respectively. This suggests that last-minute bookings and premium flexibility in OTA and direct bookings result in higher prices, potentially due to the willingness of customers to pay for convenience and certainty.

The Corporate and Group bookings at lower ADRs might indicate that these segments benefit from bulk discounts or negotiated rates, which are common in the business and group travel sectors. Targeting individual leisure travelers could present an opportunity for premium pricing.

Room Type Analysis shows that Room Type A generates the highest revenue (5.18M dollars), significantly outperforming other types. This could be due to higher demand, premium pricing, or better availability. Room type E, in contrast, generated only 755K dollars, suggesting either limited availability, lower customer interest, or possibly suboptimal marketing or positioning.

<span style="font-size: 18px;">Geographical Insights:

Portugal (PRT), United Kingdom (GBR), and France (FRA) emerge as key contributors to overall revenue, with Portugal particularly standing out due to its domestic tourism and a strong local customer base. Marketing campaigns targeting these regions could leverage localized strategies and loyalty programs to foster repeat business.
United Kingdom (GBR) stands out due to strong historical ties, ease of travel, and high-spending British tourists. Targeting promotions during peak seasons could effectively capture this market's attention.
France (FRA), with its steady inbound tourism, presents an opportunity to introduce tailored services (e.g., French-language services, travel packages) to increase engagement.

<span style="font-size: 18px;">Cancellation Impact:

Lost revenue due to cancellations and no-shows represents nearly 30% of potential earnings (2.79M dollars). Addressing this issue through better booking policies, stricter cancellation terms, or targeted rebooking campaigns could help recover a significant portion of this lost revenue.

<span style="font-size: 18px;">Lead Time and Pricing Sensitivity:

Longer lead times correlate with lower ADRs. Bookings made more than a year in advance tend to command lower prices (82.53 dollars), suggesting that price-sensitive customers book early to secure lower rates. In contrast, bookings made within 3-6 months command higher ADRs (113.83 dollars), indicating that customers booking closer to their stay are willing to pay higher rates for last-minute flexibility.

<span style="font-size: 21px;">**Overall Business Recommendations**

- Dynamic Pricing Strategy
- Leverage OTA and Direct Bookings
- Optimize Room Type Revenue
- Geographic Focus and Targeting
- Address Cancellations
- Booking Lead Time Management

## Cancellation Behavior & Customer Retention

<span style="font-size: 19px;"> Overall Cancellation Rate:

In [119]:
run_sql("""
SELECT AVG(is_canceled)*100 overall_cancelation_rate
FROM hotel_demand_adr
""")

,overall_cancelation_rate
0,27.50398926944335237400


<span style="font-size: 16px;">**Result:** 

The overall cancellation rate of 27.5% is relatively high, which suggests that cancellations are a significant issue and might require attention in terms of improving customer retention or implementing policies that reduce cancellations.

<span style="font-size: 19px;"> Cancellation Rate by Lead Time Range:

In [121]:
run_sql("""
SELECT 
CASE 
    WHEN lead_time BETWEEN 0 AND 7 THEN '0-7 days'
    WHEN lead_time BETWEEN 8 AND 30 THEN '8-30 days'
    WHEN lead_time BETWEEN 31 AND 90 THEN '1-3 Months'
    WHEN lead_time BETWEEN 90 AND 180 THEN '3-6 Months'
    WHEN lead_time BETWEEN 180 AND 365 THEN '6-12 Months'
    ELSE '1+ Year'
END AS lead_time_range,
AVG(is_canceled)*100 AS cancellation_rate
FROM hotel_demand_adr
GROUP BY lead_time_range
ORDER BY cancellation_rate DESC;
""")

,lead_time_range,cancellation_rate
0,6-12 Months,39.72875226039783001800
1,1+ Year,35.96491228070175438600
2,3-6 Months,35.08781619352700762200
3,1-3 Months,32.04493189456925526300
4,8-30 days,25.32379425188109041600
5,0-7 days,8.44515914383941443900


<span style="font-size: 16px;">**Result:**

Longer lead times (6-12 months and 1+ year) are associated with higher cancellation rates, suggesting that the further in advance a booking is made, the more likely it is to be canceled. This could be due to changing travel plans or uncertainty in guests’ schedules.

Shorter lead times (0-7 days) have the lowest cancellation rates, indicating that guests who book closer to their stay are more committed to their plans.

<span style="font-size: 19px;">  Market Segment & Cancellation Correlation:

In [181]:
run_sql("""
SELECT market_segment, SUM(is_canceled) as cancellations, COUNT(market_segment) total_reservations, AVG(is_canceled)*100 cancellation_rate
FROM hotel_demand_adr
GROUP BY market_segment
ORDER BY cancellations DESC
LIMIT 5;
""")

,market_segment,cancellations,total_reservations,cancellation_rate
0,Online TA,18156,51291,35.39802304497865122500
1,Offline TA/TO,2020,13742,14.69946150487556396400
2,Direct,1714,11655,14.70613470613470613500
3,Groups,1266,4784,26.46321070234113712400
4,Corporate,498,4106,12.12859230394544568900


<span style="font-size: 16px;">**Result:**

The "Online TA" segment has the highest cancellation rate (35.4%), which could suggest that bookings made through online travel agencies have higher rates of last-minute cancellations, potentially due to price sensitivity or a lack of loyalty to the brand.

<span style="font-size: 19px;">  Percentage of Repeated Guests:

In [125]:
run_sql("""
SELECT 
    AVG(is_repeated_guest)*100 AS repeat_guest_percentage
FROM hotel_demand_adr;
""")

,repeat_guest_percentage
0,4.18121690062672001100


<span style="font-size: 16px;">**Result:**

Only 4.18% of the total guests are repeat guests, highlighting the high turnover rate of customers and suggesting that retaining existing customers might be a challenge.

<span style="font-size: 19px;">  Average ADR for Repeated vs. Non-Repeated Guests:

In [127]:
run_sql("""
SELECT is_repeated_guest, AVG(adr)*100 avg_adr
FROM hotel_demand_adr
GROUP BY is_repeated_guest
""")

,is_repeated_guest,avg_adr
0,0,10803.694627
1,1,6511.188053


<span style="font-size: 16px;">**Result:**

The ADR for repeat guests is notably lower than for first-time guests (6,511.19 vs. 10,803.69). This could indicate that repeat guests may receive discounts or better rates or that first-time guests tend to book higher-value rooms, indicating a more lucrative target market for new customers.

<span style="font-size: 19px;"> Impact of Deposit Type on Cancellations:

In [130]:
run_sql("""
SELECT deposit_type, AVG(is_canceled)*100 AS cancellation_rate
FROM hotel_demand_adr
GROUP BY deposit_type
ORDER BY cancellation_rate DESC;
""")

,deposit_type,cancellation_rate
0,Non Refund,94.32404540763673890600
1,No Deposit,26.75010830240372794400
2,Refundable,24.03846153846153846200


<span style="font-size: 16px;">**Result:**

Non-refundable deposits have an extremely high cancellation rate (94.32%), which suggests that this booking condition is not effective at reducing cancellations or that customers may cancel despite the non-refundable policy, potentially due to exceptional circumstances or policy confusion.

<span style="font-size: 19px;"> Cancellation Impact on Revenue per Customer Segment

In [134]:
run_sql("""
SELECT customer_type, 
       SUM(CASE WHEN is_canceled = 1 THEN adr ELSE 0 END) AS lost_revenue,
       SUM(adr) AS total_revenue,
       SUM(CASE WHEN is_canceled = 1 THEN adr ELSE 0 END)/SUM(adr) AS cancellation_percentage
FROM hotel_demand_adr
GROUP BY customer_type
ORDER BY cancellation_percentage DESC;
""")

,customer_type,lost_revenue,total_revenue,cancellation_percentage
0,Transient,2596272.38,7842098.28,0.331069
1,Contract,49721.55,290335.05,0.171256
2,Transient-Party,145035.75,1009714.32,0.143640
3,Group,4189.58,34768.07,0.120501
4,transient,803.00,11118.43,0.072222


<span style="font-size: 16px;">**Result:**

Transient customers contribute the most to lost revenue and exhibit the highest cancellation percentage (33.1%). In contrast, "Group" customers and "Transient-Party" show much lower cancellation rates, indicating that larger groups or pre-booked parties tend to be more reliable, which can influence pricing and booking strategies.

### Key Insights & Conclusion

<span style="font-size: 18px;">Overall Cancellation Rate:

The high cancellation rate of 27.5% is an alarming indicator that demands urgent attention. The data suggests a notable discrepancy between the booking behavior and cancellation patterns across different customer segments, booking lead times, and booking conditions. Addressing cancellations is critical not only for improving revenue but also for optimizing customer retention strategies and refining pricing models.

<span style="font-size: 18px;">Longer Lead Times and Cancellations: 

Longer lead times (6-12 months and 1+ years) are associated with higher cancellation rates. This could indicate that guests who book well in advance are more likely to change their plans or experience uncertainty in their schedules. For example, travelers may face shifting work schedules, changes in family plans, or unforeseen events that lead to cancellations. This trend also points to a potential mismatch between customer booking patterns and the stability of their travel plans, which may lead to higher uncertainty for businesses.

<span style="font-size: 18px;">Shorter Lead Times and Committed Guests: 

Shorter lead times (0-7 days) show significantly lower cancellation rates, suggesting that guests who book closer to their stay are more committed to their travel plans. These last-minute bookers are typically more certain about their trips and less likely to cancel. The higher commitment level of these guests could be leveraged to introduce higher pricing or premium services, as these customers are likely willing to pay a premium for flexibility and convenience.

<span style="font-size: 18px;">Online Travel Agencies (OTA) and Last-Minute Cancellations: 

The Online Travel Agency (OTA) segment has the highest cancellation rate (35.4%), which is notably higher than other booking channels. This indicates that OTA customers may be more price-sensitive or less loyal, likely driven by booking flexibility and lower prices, which are prominent features of these platforms. The lack of strong brand loyalty among OTA customers could make them more likely to cancel when a better deal or an alternative option becomes available, especially as the stay date approaches.

<span style="font-size: 18px;">Customer Loyalty and Retention: 

Only 4.18% of total guests are repeat customers, revealing a high turnover rate and emphasizing the challenge of retaining existing clients. This low repeat guest rate signifies that current customer retention strategies are underperforming and that the hospitality business might be overly reliant on attracting new customers rather than cultivating loyalty. Moreover, the ADR for repeat guests is notably lower than for first-time guests ($6,511.19 vs. $10,803.69), which suggests that the repeat customer segment may benefit from discounts or incentives, while new customers tend to book higher-value rooms. This further highlights the need for targeted strategies that can raise the value of repeat guests and incentivize them to return more frequently.

<span style="font-size: 18px;">Non-Refundable Deposits and Their Effectiveness: 

The extremely high cancellation rate of 94.32% among guests who book non-refundable deposits raises concerns about the effectiveness of this policy. Despite the non-refundable nature of these bookings, cancellations remain exceedingly high, indicating that customers might cancel despite the financial penalties or may be unclear about the implications of such booking conditions. This raises the need for a review of non-refundable policies and perhaps a reconsideration of how these terms are communicated or enforced. Additionally, offering more flexible alternatives could reduce cancellations without significantly compromising revenue.

<span style="font-size: 18px;">Segmented Cancellation Behavior: 

Transient customers contribute the most to lost revenue, with the highest cancellation rate (33.1%). This indicates that transient customers, who likely book individually or for short-term stays, may be more prone to cancellations or no-shows. These customers are typically more price-sensitive, have less brand loyalty, and may prioritize flexibility, all of which increase the likelihood of cancellation. On the other hand, Group customers and Transient-Party customers exhibit much lower cancellation rates, suggesting that groups and pre-arranged parties are more reliable. Their lower cancellation rates present an opportunity to focus on group bookings or parties to minimize cancellations and maximize revenue predictability.

<span style="font-size: 21px;">**Overall Business Recommendations**

- Dynamic Cancellation Policies
- Loyalty Programs for Repeat Guests
- Targeted Marketing for Transient Customers
- Refine OTA Booking Strategy 
- Maximize Group and Party Bookings
- Enhance Customer Engagement and Communication
- Pricing Strategies for Last-Minute Bookings

## Customer Preferences & Special Requests

<span style="font-size: 19px;">  Most Frequently Reserved Room Types:

In [139]:
run_sql("""
SELECT reserved_room_type, COUNT(*) AS total_reservations
FROM hotel_demand_adr
GROUP BY reserved_room_type
ORDER BY total_reservations DESC
LIMIT 3;
""")

,reserved_room_type,total_reservations
0,A,56018
1,D,17310
2,E,5989


<span style="font-size: 16px;">**Result:**

Room type "A" has the highest number of reservations, while types "D" and "E" have significantly fewer bookings. This could highlight a preference for specific room types, which might be useful for inventory management or targeted promotions.

<span style="font-size: 19px;"> Mismatch Between Reserved and Assigned Rooms:

In [148]:
run_sql("""
SELECT 
    SUM(CASE WHEN reserved_room_type <> assigned_room_type THEN 1 END)::FLOAT / COUNT(*) * 100.0 AS mismatch_rate
FROM hotel_demand_adr;
""")

,mismatch_rate
0,14.885178


<span style="font-size: 16px;">**Result:**

A mismatch rate of approximately 14.89% between reserved and assigned rooms indicates a potential issue with the allocation of rooms, which could affect customer satisfaction. A significant mismatch might lead to dissatisfaction or cancellations.

<span style="font-size: 19px;"> Effect of Special Requests on Cancellation Rate:

In [179]:
# Checking column names
run_sql("""
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'hotel_demand_adr';
""")

,column_name
0,booking_id
1,hotel
2,is_canceled
3,lead_time
4,arrival_date_year
5,arrival_date_month
6,arrival_date_week_number
7,arrival_date_day_of_month
8,stays_in_weekend_nights
9,stays_in_week_nights


In [158]:
run_sql("""
SELECT total_of_special_requests, AVG(is_canceled)*100 as cancellation_rate 
FROM hotel_demand_adr
GROUP BY total_of_special_requests
ORDER BY cancellation_rate DESC;
""")

,total_of_special_requests,cancellation_rate
0,0,33.15267527675276752800
1,1,22.48393260378669445900
2,2,21.42247267759562841500
3,3,17.28070175438596491200
4,4,10.96774193548387096800
5,5,5.71428571428571428600


<span style="font-size: 16px;">**Result:** 

There is a clear downward trend in the cancellation rate as the number of special requests increases, indicating that guests who make special requests are more likely to show up for their reservation. This could suggest that special requests are linked to stronger booking intentions or commitments.

<span style="font-size: 19px;">  Special Requests for Repeated Guests vs First-Time Guests

In [178]:
run_sql("""
SELECT SUM(total_of_special_requests) scpecial_requests, COUNT(*) total_reservations, 
       SUM(total_of_special_requests) * 1.0/COUNT(*) special_requests_per_reservation, is_repeated_guest 
FROM hotel_demand_adr
GROUP BY is_repeated_guest
ORDER BY scpecial_requests DESC;
""")

,scpecial_requests,total_reservations,special_requests_per_reservation,is_repeated_guest
0,58011,82866,0.70005792484251683441,0
1,2453,3616,0.67837389380530973451,1


<span style="font-size: 16px;">**Result:** 

Non-repeat guests have a much higher total number of special requests (82,866 vs. 3,616), but when normalized by the number of reservations, the special requests per reservation are quite similar for repeat (0.68) and non-repeat guests (0.70).

Insight: While non-repeat guests make more special requests overall, the rate of special requests per reservation is almost identical across repeat and first-time guests. This suggests that the nature of guest requests might not differ significantly between repeat and first-time visitors, even though the total number of requests is larger for non-repeat guests.

### Key Insights & Conclusion

<span style="font-size: 18px;">Room Type Preferences: 

Room type "A" sees the highest number of reservations, indicating a strong preference for this type of room among guests. This preference could be influenced by factors such as location, amenities, or price. Conversely, room types "D" and "E" have significantly fewer bookings, pointing to a potential mismatch between guest preferences and room availability. This suggests that room types "D" and "E" might not align with customer needs or desires, possibly due to suboptimal marketing, less desirable room features, or pricing issues. Understanding the reasons behind these preferences can help optimize inventory management and guide targeted promotions to boost bookings for less popular room types.

<span style="font-size: 18px;">Mismatch Between Reserved and Assigned Rooms: 

The 14.89% mismatch rate between reserved and assigned rooms is a crucial issue for customer satisfaction. When a guest’s requested room type is not available upon arrival, it can lead to dissatisfaction, complaints, or even cancellations. This highlights a need for more precise room allocation processes, potentially involving better forecasting tools, improved communication between booking systems and the front desk, or more flexible room assignment protocols. Ensuring that guests receive the room type they reserved will likely improve guest satisfaction and reduce cancellation rates.

<span style="font-size: 18px;">Special Requests and Booking Commitments: 

The downward trend in the cancellation rate as the number of special requests increases is a noteworthy finding. Guests who make special requests are less likely to cancel their reservations, suggesting that special requests are indicative of stronger commitment and intention to show up. These requests could range from room preferences to specific amenities or services. The correlation between special requests and reduced cancellations implies that guests who have taken the time to make specific demands are more invested in their stays. This could be leveraged as a signal for higher booking certainty, which can inform both customer service and booking strategies.

<span style="font-size: 18px;">Special Requests by Repeat and Non-Repeat Guests: 

While non-repeat guests submit a significantly higher total number of special requests (82,866 vs. 3,616), the rate of special requests per reservation is quite similar across both repeat (0.68 requests per reservation) and non-repeat guests (0.70 requests per reservation). This indicates that the nature of special requests does not differ substantially between first-time and repeat guests, even though the total number of requests from non-repeat guests is much higher. Non-repeat guests might be more likely to book in larger numbers or through channels with higher volumes of transient bookings (e.g., OTAs), which could explain the higher overall count. For repeat guests, the similar rate of requests per booking implies that their preferences and needs remain relatively consistent with their past stays, underscoring the importance of remembering and catering to individual preferences for return visits.

<span style="font-size: 21px;">**Overall Business Recommendations**

- Optimize Room Inventory Management
- Improve Room Allocation Processes
- Leverage Special Requests for Customer Retention
- Target Marketing for Non-Repeat Guests
- Enhance Personalization for Repeat Guests
- Monitor and Analyze Special Requests Data

# Business Recommendations & Strategic Actions

<span style="font-size: 18px;">**Maximize Online Presence**

Given the dominance of OTAs, strengthening partnerships with these platforms while enhancing direct online booking channels will ensure higher visibility and attract a larger pool of potential guests.

Strategic Action: Focus on digital marketing campaigns, optimize booking platforms for ease of use, and offer incentives for direct bookings, such as discounts or exclusive benefits. 

<span style="font-size: 18px;">**Address Cancellations**

The increasing trend of cancellations is impacting occupancy rates and revenue. To address this, refining cancellation policies is crucial.

Strategic Action: Consider implementing non-refundable booking options, offering flexible cancellation alternatives (e.g., partial refunds for earlier cancellations), and using customer behavior analysis to anticipate cancellations. Additionally, introduce last-minute offers to fill rooms at premium rates.

<span style="font-size: 18px;">**Target Seasonal Demand**

Seasonality is a key factor in demand fluctuations. Strategic planning for peak and off-peak months can help smoothen revenue streams throughout the year.

Strategic Action: Create seasonal packages, special promotions, and loyalty offers for peak periods (e.g., summer months) and targeted campaigns with discounts during off-peak seasons (e.g., fall and winter).

<span style="font-size: 18px;">**Optimize Weekend & Weekday Pricing**

There is potential to capitalize on both weekend and weekday bookings through tailored pricing strategies.

Strategic Action: Implement dynamic pricing with special weekend promotions to attract more weekend guests, while offering weekday deals to fill vacancies and maintain occupancy.

<span style="font-size: 18px;">**Promote Extended Stays During Peak Seasons**

Longer stays are more common during peak seasons, indicating an opportunity to further attract customers with longer vacation needs.

Strategic Action: Offer discounted rates or special packages for extended stays during the peak months to boost revenue during these high-demand periods.

<span style="font-size: 18px;">**Leverage Booking Lead Times**

The trend of longer booking lead times provides an opportunity to secure advance bookings.

Strategic Action: Create early booking promotions to incentivize guests to book in advance, ensuring higher occupancy during peak periods and securing revenue early.

<span style="font-size: 18px;">**Dynamic Pricing Strategy**

Pricing flexibility is essential for optimizing revenue during both peak and off-peak periods.

Strategic Action: Implement aggressive pricing during peak months (June-August) and create attractive offers during off-peak months to maintain occupancy levels year-round.

<span style="font-size: 18px;">**Optimize Room Type Revenue**

Room types like "A" are in high demand, while others (e.g., "E") are underperforming.

Strategic Action: Prioritize popular room types, investigate underperforming rooms, and improve marketing, pricing, or amenities for those room types. Consider introducing premium options for high-demand rooms during peak seasons.

<span style="font-size: 18px;">**Geographic Focus and Targeting**

Different regions may have unique customer preferences and behaviors.

Strategic Action: Develop targeted marketing campaigns for key geographic markets (e.g., Portugal, UK, France), incorporating language preferences and cultural relevance to better engage potential customers.

<span style="font-size: 18px;">**Enhance Customer Retention through Loyalty Programs**

Repeat guests tend to generate higher ADRs, so enhancing retention is essential for long-term revenue growth.

Strategic Action: Develop or enhance loyalty programs offering repeat guests benefits such as discounts, room upgrades, or exclusive offers, which could boost their likelihood of booking again and improve customer satisfaction.

<span style="font-size: 18px;">**Maximize Group and Party Bookings**

Group bookings exhibit lower cancellation rates, offering a more reliable source of revenue.

Strategic Action: Promote group bookings with tailored packages and incentives, such as discounts or added perks for large parties, to secure predictable revenue.

<span style="font-size: 18px;">**Improve Room Inventory Management**

Room type demand does not align perfectly with available inventory, particularly for less popular room types.

Strategic Action: Align room inventory more effectively with demand, particularly for room type "A." Investigate the reasons behind low demand for other room types and adjust their pricing, marketing, or amenities to increase their appeal.

<span style="font-size: 18px;">**Improve Room Allocation Processes**

Room mismatches between reservations and assignments could lead to customer dissatisfaction.

Strategic Action: Enhance room allocation accuracy by implementing real-time room tracking systems and automating assignments. Ensure that guests are assigned the room they reserved to improve satisfaction and reduce cancellations.

<span style="font-size: 18px;">**Leverage Special Requests for Customer Retention**

Special requests correlate with reduced cancellations, indicating that they can be used to strengthen guest loyalty.

Strategic Action: Encourage guests to make special requests during the booking process and offer incentives, such as discounts, for early special requests to enhance customer commitment.

<span style="font-size: 18px;">**Refine OTA Booking Strategy**

OTA customers show higher cancellation rates, indicating the need for a tailored approach.

Strategic Action: Develop loyalty-building strategies for OTA guests, such as offering exclusive perks for direct bookings or creating special promotions to encourage OTA guests to commit to their bookings.

<span style="font-size: 18px;">**Monitor Special Requests Data**

Special requests provide valuable insights into customer preferences.

Strategic Action: Continuously monitor and analyze special requests data to tailor offerings and improve customer service. Consider offering personalized packages based on frequent guest requests to enhance customer experience.


By implementing these recommendations, businesses can address key issues such as cancellations, optimize pricing and inventory management, and focus on customer engagement and retention. This strategic approach will drive revenue growth, improve guest satisfaction, and ensure sustainable long-term success.

**Closing Connection**

In [35]:
#Closing Connection 
cursor.close()
conn.close()